# World Cup 2026 Winner - GROUPED_BINARY Market Simulation

This notebook simulates a **GROUPED_BINARY** prediction market with:
- **5 separate YES/NO markets** (one per country)
- Each market asks: "Will [Country] win the World Cup?"
- Each starts at 50% YES / 50% NO
- **Total YES probability = 250%** → Creates arbitrage opportunity!
- **10 Traders** making bets (including NO bets)
- **1 Platform** (receives 2% of winner fee)
- **1 Market Creator** (receives 2% of winner fee)
- **Market Resolution** with fee distribution to ALL winners (YES on winner + NO on losers)

## Participants
1. Platform (Nostra)
2. Market Creator (Jay)
3-12. Traders (Alice, Bob, Carol, Dave, Eve, Frank, Grace, Henry, Iris, Jack)

In [4]:
# Install web3 if not already installed
!pip install web3

zsh:1: command not found: pip


In [1]:
# Minimal ERC-20 ABI for balanceOf and decimals
ERC20_ABI = [
    {
      "inputs": [],
      "stateMutability": "nonpayable",
      "type": "constructor"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "spender",
          "type": "address"
        },
        {
          "internalType": "uint256",
          "name": "allowance",
          "type": "uint256"
        },
        {"internalType": "uint256",
          "name": "needed",
          "type": "uint256"
        }
      ],
      "name": "ERC20InsufficientAllowance",
      "type": "error"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "sender",
          "type": "address"
        },
        {
          "internalType": "uint256",
          "name": "balance",
          "type": "uint256"
        },
        {
          "internalType": "uint256",
          "name": "needed",
          "type": "uint256"
        }
      ],
      "name": "ERC20InsufficientBalance",
      "type": "error"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "approver",
          "type": "address"
        }
      ],
      "name": "ERC20InvalidApprover",
      "type": "error"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "receiver",
          "type": "address"
        }
      ],
      "name": "ERC20InvalidReceiver",
      "type": "error"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "sender",
          "type": "address"
        }
      ],
      "name": "ERC20InvalidSender",
      "type": "error"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "spender",
          "type": "address"
        }
      ],
      "name": "ERC20InvalidSpender",
      "type": "error"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "owner",
          "type": "address"
        }
      ],
      "name": "OwnableInvalidOwner",
      "type": "error"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "OwnableUnauthorizedAccount",
      "type": "error"
    },
    {
      "anonymous": False,
      "inputs": [
        {
          "indexed": True,
          "internalType": "address",
          "name": "owner",
          "type": "address"
        },
        {
          "indexed": True,
          "internalType": "address",
          "name": "spender",
          "type": "address"
        },
        {
          "indexed": False,
          "internalType": "uint256",
          "name": "value",
          "type": "uint256"
        }
      ],
      "name": "Approval",
      "type": "event"
    },
    {
      "anonymous": False,
      "inputs": [
        {
          "indexed": True,
          "internalType": "address",
          "name": "previousOwner",
          "type": "address"
        },
        {
          "indexed": True,
          "internalType": "address",
          "name": "newOwner",
          "type": "address"
        }
      ],
      "name": "OwnershipTransferred",
      "type": "event"
    },
    {
      "anonymous": False,
      "inputs": [
        {
          "indexed": True,
          "internalType": "address",
          "name": "from",
          "type": "address"
        },
        {
          "indexed": True,
          "internalType": "address",
          "name": "to",
          "type": "address"
        },
        {
          "indexed": False,
          "internalType": "uint256",
          "name": "value",
          "type": "uint256"
        }
      ],
      "name": "Transfer",
      "type": "event"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "owner",
          "type": "address"
        },
        {
          "internalType": "address",
          "name": "spender",
          "type": "address"
        }
      ],
      "name": "allowance",
      "outputs": [
        {
          "internalType": "uint256",
          "name": "",
          "type": "uint256"
        }
      ],
      "stateMutability": "view",
      "type": "function"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "spender",
          "type": "address"
        },
        {
          "internalType": "uint256",
          "name": "value",
          "type": "uint256"
        }
      ],
      "name": "approve",
      "outputs": [
        {
          "internalType": "bool",
          "name": "",
          "type": "bool"
        }
      ],
      "stateMutability": "nonpayable",
      "type": "function"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "balanceOf",
      "outputs": [
        {
          "internalType": "uint256",
          "name": "",
          "type": "uint256"
        }
      ],
      "stateMutability": "view",
      "type": "function"
    },
    {
      "inputs": [],
      "name": "decimals",
      "outputs": [
        {
          "internalType": "uint8",
          "name": "",
          "type": "uint8"
        }
      ],
      "stateMutability": "pure",
      "type": "function"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "to",
          "type": "address"
        },
        {
          "internalType": "uint256",
          "name": "amount",
          "type": "uint256"
        }
      ],
      "name": "mint",
      "outputs": [],
      "stateMutability": "nonpayable",
      "type": "function"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "to",
          "type": "address"
        },
        {
          "internalType": "uint256",
          "name": "amount",
          "type": "uint256"
        }
      ],
      "name": "mintForTesting",
      "outputs": [],
      "stateMutability": "nonpayable",
      "type": "function"
    },
    {
      "inputs": [],
      "name": "name",
      "outputs": [
        {
          "internalType": "string",
          "name": "",
          "type": "string"
        }
      ],
      "stateMutability": "view",
      "type": "function"
    },
    {
      "inputs": [],
      "name": "owner",
      "outputs": [
        {
          "internalType": "address",
          "name": "",
          "type": "address"
        }
      ],
      "stateMutability": "view",
      "type": "function"
    },
    {
      "inputs": [],
      "name": "renounceOwnership",
      "outputs": [],
      "stateMutability": "nonpayable",
      "type": "function"
    },
    {
      "inputs": [],
      "name": "symbol",
      "outputs": [
        {
          "internalType": "string",
          "name": "",
          "type": "string"
        }
      ],
      "stateMutability": "view",
      "type": "function"
    },
    {
      "inputs": [],
      "name": "totalSupply",
      "outputs": [
        {
          "internalType": "uint256",
          "name": "",
          "type": "uint256"
        }
      ],
      "stateMutability": "view",
      "type": "function"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "to",
          "type": "address"
        },
        {
          "internalType": "uint256",
          "name": "value",
          "type": "uint256"
        }
      ],
      "name": "transfer",
      "outputs": [
        {
          "internalType": "bool",
          "name": "",
          "type": "bool"
        }
      ],
      "stateMutability": "nonpayable",
      "type": "function"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "from",
          "type": "address"
        },
        {
          "internalType": "address",
          "name": "to",
          "type": "address"
        },
        {
          "internalType": "uint256",
          "name": "value",
          "type": "uint256"
        }
      ],
      "name": "transferFrom",
      "outputs": [
        {
          "internalType": "bool",
          "name": "",
          "type": "bool"
        }
      ],
      "stateMutability": "nonpayable",
      "type": "function"
    },
    {
      "inputs": [
        {
          "internalType": "address",
          "name": "newOwner",
          "type": "address"
        }
      ],
      "name": "transferOwnership",
      "outputs": [],
      "stateMutability": "nonpayable",
      "type": "function"
    }
  ]


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from collections import defaultdict

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("📊 World Cup 2026 Winner - GROUPED_BINARY Market Simulation")
print("=" * 60)

📊 World Cup 2026 Winner - GROUPED_BINARY Market Simulation


## 1. Market Setup - GROUPED_BINARY Structure

**Market Group**: "Who will win the 2026 FIFA World Cup?"

**Individual Binary Markets** (5 separate markets):
1. "Will Brazil win?" → YES 50% / NO 50% 🇧🇷
2. "Will Argentina win?" → YES 50% / NO 50% 🇦🇷
3. "Will France win?" → YES 50% / NO 50% 🇫🇷
4. "Will Germany win?" → YES 50% / NO 50% 🇩🇪
5. "Will Spain win?" → YES 50% / NO 50% 🇪🇸

**Key Insight**: Sum of YES probabilities = 5 × 50% = **250%** (Should be ~100%!)

This creates a **massive arbitrage opportunity** that the hybrid bot should detect.

In [3]:
SERVER_WALLET_PRIVATE_KEY='396f06f70b40af351a70230b5c46f30b36988454415fe98f6cf7dae51c23d77c'
TRADER_1_PRIVATE_KEY='eadb202559314eaa99ce549807bcdb212350931dccf487c558dff7e8d0bc22a1'
TRADER_2_PRIVATE_KEY='7ed471ef263131adfd5888ec44b26dc684ac7c3e1abb155f4ddb560dfd3ff61f'
TRADER_3_PRIVATE_KEY='d2a9dd4826408e43bf604d12b07f6ddd58cb564c1fe32690f76c618e327c0787'
TRADER_4_PRIVATE_KEY='a46ff853fefc955027aec5dd855956dc9cac83bb3a8ea0733178760ba2782f5c'
TRADER_5_PRIVATE_KEY='f878a504dc35fed49c22a6513d0bf866fac66e23303a7c7ea59705f02e5fac76'
TRADER_6_PRIVATE_KEY='02d42fa943aa707dc0343b373fded65d91a9daea4266823754010663edb9dcb6'
TRADER_7_PRIVATE_KEY='0a38483104f3d8a7f3530058a4ff85f39ff7e0a578dad44b6be533a735b0f584'
TRADER_8_PRIVATE_KEY='24be1e72dd72d1590c98b0f4a4e108865a99b72bbd038e781cf34144920ea468'
TRADER_9_PRIVATE_KEY='03c779dea2dd048083f7f83119f2c8bb2b44b096eca5d76f1dc6208496762132'
TRADER_10_PRIVATE_KEY='b6b7f01d44648c34b6aed589ec9dcb4d921add2c595fad1eef03cd76eae2eb3d'

RRPC_URL="https://bnb-testnet.g.alchemy.com/v2/0nZajqTds59iJ3zvHZKL4"
WS_RPC_URL="wss://bnb-testnet.g.alchemy.com/v2/0nZajqTds59iJ3zvHZKL4"

# Private keys defined as constants in the notebook.

from web3 import Web3
from eth_account import Account
import time # Added for potential sleep

# Connect to BNB testnet
w3 = Web3(Web3.HTTPProvider(RRPC_URL))

if w3.is_connected():
    print("✅ Connected to Web3 provider.")
else:
    print("❌ Failed to connect to Web3 provider.")

mUSDC_CONTRACT_ADDRESS = "0xA9dEfa17DB340188d5a637a23619e13b270d70F9"

# The ERC20_ABI is defined in cell zhKbO1mJLLPx and is used here.
# DO NOT REDEFINE ERC20_ABI here with a minimal version, as it omits the transfer function.

musdc_contract = w3.eth.contract(address=w3.to_checksum_address(mUSDC_CONTRACT_ADDRESS), abi=ERC20_ABI)

# Get decimals for correct balance formatting
decimals = musdc_contract.functions.decimals().call()

private_keys = {
    'SERVER_WALLET': SERVER_WALLET_PRIVATE_KEY,
    'TRADER_1': TRADER_1_PRIVATE_KEY,
    'TRADER_2': TRADER_2_PRIVATE_KEY,
    'TRADER_3': TRADER_3_PRIVATE_KEY,
    'TRADER_4': TRADER_4_PRIVATE_KEY,
    'TRADER_5': TRADER_5_PRIVATE_KEY,
    'TRADER_6': TRADER_6_PRIVATE_KEY,
    'TRADER_7': TRADER_7_PRIVATE_KEY,
    'TRADER_8': TRADER_8_PRIVATE_KEY,
    'TRADER_9': TRADER_9_PRIVATE_KEY,
    'TRADER_10': TRADER_10_PRIVATE_KEY,
}

print(f"\n💰 Initial mUSDC Token Balances on {mUSDC_CONTRACT_ADDRESS} (BNB Testnet):")
print("=" * 70)
initial_balances = {}
trader_addresses = {} # Store addresses to avoid re-calculating
for name, private_key in private_keys.items():
    try:
        account = Account.from_key(private_key)
        address = account.address
        trader_addresses[name] = address
        balance_wei = musdc_contract.functions.balanceOf(address).call()
        balance_token = balance_wei / (10**decimals)
        initial_balances[name] = balance_token
        print(f"{name:<15} Address: {address:<42} Balance: {balance_token:,.6f} mUSDC")
    except Exception as e:
        print(f"Error processing {name} for initial balance: {e}")
print("=" * 70)

# -------------------------------------------------------------
# Token Distribution Logic
# -------------------------------------------------------------
print("\n🔄 Adjusting trader balances to 1000 mUSDC if needed...")
print("=" * 70)

server_account = Account.from_key(SERVER_WALLET_PRIVATE_KEY)
server_address = server_account.address
TARGET_BALANCE = 1000.0
TRADER_1_NAME = "TRADER_1"

def to_wei_amount(token_amount: float) -> int:
    """Convert human-readable token amount to smallest unit with safe rounding."""
    return int(round(token_amount * (10 ** decimals)))

def send_erc20(from_privkey: str, from_address: str, to_address: str, amount_wei: int, nonce: int | None = None):
    """Send ERC20 tokens with basic error handling. Returns (success, tx_hash_or_msg, used_nonce)."""
    try:
        gas_price = w3.eth.gas_price

        tx_body = {
            "from": from_address,
            "gasPrice": gas_price,
        }

        # nonce 관리: 지정되면 그대로 사용, 아니면 현재 nonce 조회
        if nonce is not None:
            tx_body["nonce"] = nonce
        else:
            tx_body["nonce"] = w3.eth.get_transaction_count(from_address)

        # 가스 추정 (실패 시 fallback 100000)
        try:
            tx_for_estimate = musdc_contract.functions.transfer(
                to_address,
                amount_wei
            ).build_transaction(tx_body)
            estimated_gas = w3.eth.estimate_gas(tx_for_estimate)
            tx_body["gas"] = int(estimated_gas * 1.2)  # 20% 여유
        except Exception as e:
            print(f"      ⚠️ Gas estimation failed, using default 100000. Reason: {e}")
            tx_body["gas"] = 100000

        # 최종 트랜잭션
        tx = musdc_contract.functions.transfer(
            to_address,
            amount_wei
        ).build_transaction(tx_body)

        signed_tx = w3.eth.account.sign_transaction(tx, private_key=from_privkey)

        # ✅ web3.py v6+: raw_transaction
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
        print(f"      ⏱  Sent tx from {from_address} to {to_address}, amount_wei={amount_wei}. Tx Hash: {tx_hash.hex()}")

        receipt = w3.eth.wait_for_transaction_receipt(tx_hash, timeout=300)
        if receipt.status == 1:
            print(f"      🎉 Tx confirmed in block {receipt.blockNumber}.")
            return True, tx_hash.hex(), tx["nonce"]
        else:
            print(f"      ⚠️ Tx reverted with status {receipt.status}.")
            return False, f"reverted (status={receipt.status})", tx["nonce"]

    except Exception as e:
        print(f"      ❌ Error sending tx: {e}")
        # nonce를 알 수 없을 수도 있으므로 None 반환
        return False, str(e), None


# 1️⃣ TRADER_1 잔고를 1000으로 줄이기 (서버로 전송)
if TRADER_1_NAME in trader_addresses:
    trader1_address = trader_addresses[TRADER_1_NAME]
    current_balance_trader1 = initial_balances[TRADER_1_NAME]

    if current_balance_trader1 > TARGET_BALANCE:
        amount_to_send_token = current_balance_trader1 - TARGET_BALANCE
        amount_to_send_wei = to_wei_amount(amount_to_send_token)

        if amount_to_send_wei > 0:
            print(f"   🔽 {TRADER_1_NAME} ({trader1_address}) balance {current_balance_trader1:,.6f} > {TARGET_BALANCE}.")
            print(f"      → Sending {amount_to_send_token:,.6f} mUSDC from {TRADER_1_NAME} to SERVER_WALLET ({server_address})")

            trader1_account = Account.from_key(private_keys[TRADER_1_NAME])
            ok, msg, _ = send_erc20(
                from_privkey=private_keys[TRADER_1_NAME],
                from_address=trader1_account.address,
                to_address=server_address,
                amount_wei=amount_to_send_wei,
                nonce=None  # 트레이더1은 단일 tx라 그냥 현재 nonce 사용
            )
            if ok:
                # 메모리 상 잔고 업데이트
                initial_balances[TRADER_1_NAME] = TARGET_BALANCE
            else:
                print(f"   ⚠️ Failed to send from {TRADER_1_NAME}: {msg}")
            time.sleep(1)
    else:
        print(f"   ✅ {TRADER_1_NAME} ({trader1_address}) balance is {current_balance_trader1:,.6f} (<= {TARGET_BALANCE}). No tokens to send to server.")
else:
    print(f"   ⚠️ {TRADER_1_NAME} not found in trader_addresses.")


# 2️⃣ 서버가 나머지 트레이더들을 1000으로 맞춰주기
# 서버 nonce를 한 번만 가져와서 로컬에서 증가시키며 사용
server_nonce = w3.eth.get_transaction_count(server_address)

for name, private_key in private_keys.items():
    if name in ["SERVER_WALLET", TRADER_1_NAME]:
        continue  # 서버와 TRADER_1은 이미 처리했거나 처리 대상 아님

    trader_address = trader_addresses[name]
    current_balance = initial_balances[name]

    if current_balance < TARGET_BALANCE:
        amount_to_top_up_token = TARGET_BALANCE - current_balance
        amount_to_top_up_wei = to_wei_amount(amount_to_top_up_token)

        if amount_to_top_up_wei <= 0:
            print(f"   ✅ {name} ({trader_address}) current balance {current_balance:,.6f} mUSDC. No positive amount to top up needed.")
            continue

        # 서버 잔고 / 가스 체크
        server_mUSDC_balance_wei = musdc_contract.functions.balanceOf(server_address).call()
        server_bnb_balance_wei = w3.eth.get_balance(server_address)
        estimated_gas_cost = w3.eth.gas_price * 100000

        if server_mUSDC_balance_wei < amount_to_top_up_wei:
            print(f"❌ SERVER_WALLET ({server_address}) has insufficient mUSDC to top up {name} ({trader_address}).")
            print(f"   Needed: {amount_to_top_up_token:,.6f} mUSDC, Server has: {server_mUSDC_balance_wei / (10**decimals):,.6f} mUSDC.")
            continue

        if server_bnb_balance_wei < estimated_gas_cost:
            print(f"❌ SERVER_WALLET ({server_address}) has insufficient BNB for gas to top up {name} ({trader_address}).")
            print(f"   Needed: {w3.from_wei(estimated_gas_cost, 'ether'):.8f} BNB, Server has: {w3.from_wei(server_bnb_balance_wei, 'ether'):.8f} BNB.")
            continue

        print(f"   🔼 Topping up {name} ({trader_address}) from {current_balance:,.6f} → {TARGET_BALANCE:,.6f} mUSDC")
        ok, msg, used_nonce = send_erc20(
            from_privkey=SERVER_WALLET_PRIVATE_KEY,
            from_address=server_address,
            to_address=trader_address,
            amount_wei=amount_to_top_up_wei,
            nonce=server_nonce
        )

        # nonce 업데이트: 전송을 시도했으면, (성공/실패와 상관 없이) next nonce로 증가
        if used_nonce is not None:
            server_nonce = used_nonce + 1

        if not ok:
            print(f"   ⚠️ Failed to top up {name}: {msg}")
        time.sleep(1)

    else:
        print(f"   ✅ {name} ({trader_address}) already has {current_balance:,.6f} mUSDC (>= {TARGET_BALANCE}). No top-up needed.")

print("=" * 70)


✅ Connected to Web3 provider.

💰 Initial mUSDC Token Balances on 0xA9dEfa17DB340188d5a637a23619e13b270d70F9 (BNB Testnet):
SERVER_WALLET   Address: 0x440d17f23A9e6983a7447C4A9b06109bb0aAA8bE Balance: 999,946,000.000000 mUSDC
TRADER_1        Address: 0x6eDF62fc2dbDef1C050E295679a2fA8051F11621 Balance: 1,000.000000 mUSDC
TRADER_2        Address: 0x140097D5513587B222eE833587e939d697c31e1B Balance: 1,000.000000 mUSDC
TRADER_3        Address: 0x2572F01F04F6f32d658f257f433a4673CBd12c85 Balance: 1,000.000000 mUSDC
TRADER_4        Address: 0x596611721Caa376cBD1Ef040d7f9f5883C919f02 Balance: 1,000.000000 mUSDC
TRADER_5        Address: 0x6af487fc110cdCB80508994b9D982ba6e455f39A Balance: 1,000.000000 mUSDC
TRADER_6        Address: 0x1cfF2d41596f019b7aeb5F2F3a93224991A6B03F Balance: 1,000.000000 mUSDC
TRADER_7        Address: 0xfeaD659C4efeB8C056773f49a9420F95a67479d7 Balance: 1,000.000000 mUSDC
TRADER_8        Address: 0xfA19034fEb25A6E07DF13dAC19Ac622b9dFf3e06 Balance: 1,000.000000 mUSDC
TRADER_

In [4]:
# -------------------------------------------------------------
# Final Balance Display (after top-up)
# -------------------------------------------------------------
print(f"\n💰 Final mUSDC Token Balances on {mUSDC_CONTRACT_ADDRESS} (BNB Testnet) after adjustments:")
print("=" * 70)
for name, private_key in private_keys.items():
    try:
        account = Account.from_key(private_key)
        address = account.address
        balance_wei = musdc_contract.functions.balanceOf(address).call()
        balance_token = balance_wei / (10**decimals)
        print(f"{name:<15} Address: {address:<42} Balance: {balance_token:,.6f} mUSDC")
    except Exception as e:
        print(f"Error processing {name} for final balance: {e}")
print("=" * 70)


💰 Final mUSDC Token Balances on 0xA9dEfa17DB340188d5a637a23619e13b270d70F9 (BNB Testnet) after adjustments:
SERVER_WALLET   Address: 0x440d17f23A9e6983a7447C4A9b06109bb0aAA8bE Balance: 999,946,000.000000 mUSDC
TRADER_1        Address: 0x6eDF62fc2dbDef1C050E295679a2fA8051F11621 Balance: 1,000.000000 mUSDC
TRADER_2        Address: 0x140097D5513587B222eE833587e939d697c31e1B Balance: 1,000.000000 mUSDC
TRADER_3        Address: 0x2572F01F04F6f32d658f257f433a4673CBd12c85 Balance: 1,000.000000 mUSDC
TRADER_4        Address: 0x596611721Caa376cBD1Ef040d7f9f5883C919f02 Balance: 1,000.000000 mUSDC
TRADER_5        Address: 0x6af487fc110cdCB80508994b9D982ba6e455f39A Balance: 1,000.000000 mUSDC
TRADER_6        Address: 0x1cfF2d41596f019b7aeb5F2F3a93224991A6B03F Balance: 1,000.000000 mUSDC
TRADER_7        Address: 0xfeaD659C4efeB8C056773f49a9420F95a67479d7 Balance: 1,000.000000 mUSDC
TRADER_8        Address: 0xfA19034fEb25A6E07DF13dAC19Ac622b9dFf3e06 Balance: 1,000.000000 mUSDC
TRADER_9        Addre

## 2. Trading Activity

Traders buy YES shares for countries they think will win.
**Some smart traders also buy NO shares** on countries they think will lose!
Each trade affects the YES/NO prices in that specific binary market.

In [8]:
# Market Configuration
COUNTRIES = ['Brazil', 'Argentina', 'France', 'Germany', 'Spain']
INITIAL_YES_PRICE = 0.50  # Each market starts at 50% YES
INITIAL_NO_PRICE = 0.50   # Each market starts at 50% NO
WINNER_FEE_RATE = 0.04  # 4% total fee
PLATFORM_FEE_SHARE = 0.50  # Platform gets 50% of 4% = 2%
CREATOR_FEE_SHARE = 0.50   # Creator gets 50% of 4% = 2%


# Initialize market state - GROUPED_BINARY structure
# Each country has its own binary market with YES/NO outcomes
market_state = {}
for country in COUNTRIES:
    market_state[country] = {
        'yes_price': INITIAL_YES_PRICE,
        'no_price': INITIAL_NO_PRICE,
        'volume': 0,
    }

# Participants
TRADERS = ['Alice', 'Bob', 'Carol', 'Dave', 'Eve', 'Frank', 'Grace', 'Henry', 'Iris', 'Jack']
PLATFORM = 'Nostra Platform'
CREATOR = 'Jay (Market Creator)'

# Initialize balances
balances = {
    PLATFORM: {'cash': 0, 'shares': {}},
    CREATOR: {'cash': 0, 'shares': {}},
}

for trader in TRADERS:
    balances[trader] = {
        'cash': 1000,  # Each trader starts with $1000
        'shares': {}
    }

# Initialize shares for each country's YES/NO
for participant in [PLATFORM, CREATOR] + TRADERS:
    for country in COUNTRIES:
        balances[participant]['shares'][f'{country}_YES'] = 0
        balances[participant]['shares'][f'{country}_NO'] = 0

print(f"✅ Market Group initialized with {len(COUNTRIES)} binary markets")
print(f"\n📊 Market Structure (GROUPED_BINARY):")
for country in COUNTRIES:
    yes_prob = market_state[country]['yes_price'] * 100
    no_prob = market_state[country]['no_price'] * 100
    print(f"   '{country} wins?'  →  YES: {yes_prob:.0f}%  |  NO: {no_prob:.0f}%")

total_yes_prob = sum(market_state[c]['yes_price'] for c in COUNTRIES) * 100
print(f"\n⚠️  Total YES probability: {total_yes_prob:.0f}% (Should be ~100%!)")
print(f"\n✅ {len(TRADERS)} traders ready with $1,000 each")
print(f"\nFee Structure:")
print(f"  - Total Winner Fee: {WINNER_FEE_RATE*100}%")
print(f"  - Platform Share: {PLATFORM_FEE_SHARE*100}% of fee = {WINNER_FEE_RATE*PLATFORM_FEE_SHARE*100}% total")
print(f"  - Creator Share: {CREATOR_FEE_SHARE*100}% of fee = {WINNER_FEE_RATE*CREATOR_FEE_SHARE*100}% total")


# Trade history
trades = []
price_history = defaultdict(list)

def record_prices(timestamp):
    """Record current YES prices for all markets"""
    for country in COUNTRIES:
        price_history[country].append({
            'timestamp': timestamp,
            'yes_price': market_state[country]['yes_price'],
            'yes_probability': market_state[country]['yes_price'] * 100,
            'no_price': market_state[country]['no_price'],
        })

def execute_trade(trader, country, side, shares, price, timestamp):
    """
    Execute a trade
    side: 'YES' or 'NO'
    """
    cost = shares * price

    if balances[trader]['cash'] < cost:
        print(f"❌ {trader} insufficient funds")
        return False

    # Update balances
    balances[trader]['cash'] -= cost
    balances[trader]['shares'][f'{country}_{side}'] += shares

    # Record trade
    trades.append({
        'timestamp': timestamp,
        'trader': trader,
        'country': country,
        'side': side,
        'shares': shares,
        'price': price,
        'cost': cost,
    })

    # Update market volume
    market_state[country]['volume'] += cost

    # Price impact based on trade size
    price_impact = 0.02 * (cost / 100)  # 2% per $100 traded

    if side == 'YES':
        # Buying YES increases YES price, decreases NO price
        market_state[country]['yes_price'] = min(0.95, market_state[country]['yes_price'] + price_impact)
        market_state[country]['no_price'] = max(0.05, 1 - market_state[country]['yes_price'])
    else:
        # Buying NO increases NO price, decreases YES price
        market_state[country]['no_price'] = min(0.95, market_state[country]['no_price'] + price_impact)
        market_state[country]['yes_price'] = max(0.05, 1 - market_state[country]['no_price'])

    return True

# Simulate trading over 30 days
base_time = datetime.now()
record_prices(base_time)

print("📈 Starting trading simulation...\n")



✅ Market Group initialized with 5 binary markets

📊 Market Structure (GROUPED_BINARY):
   'Brazil wins?'  →  YES: 50%  |  NO: 50%
   'Argentina wins?'  →  YES: 50%  |  NO: 50%
   'France wins?'  →  YES: 50%  |  NO: 50%
   'Germany wins?'  →  YES: 50%  |  NO: 50%
   'Spain wins?'  →  YES: 50%  |  NO: 50%

⚠️  Total YES probability: 250% (Should be ~100%!)

✅ 10 traders ready with $1,000 each

Fee Structure:
  - Total Winner Fee: 4.0%
  - Platform Share: 50.0% of fee = 2.0% total
  - Creator Share: 50.0% of fee = 2.0% total
📈 Starting trading simulation...



In [ ]:
print("\n📅 DAY 1: Brazil Enthusiasm")
print("-" * 60)

day = 1
timestamp = base_time + timedelta(days=day)

execute_trade('Alice', 'Brazil', 'YES', 100, market_state['Brazil']['yes_price'], timestamp)
execute_trade('Bob', 'Brazil', 'YES', 150, market_state['Brazil']['yes_price'], timestamp)
execute_trade('Carol', 'Brazil', 'YES', 80, market_state['Brazil']['yes_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 3 trades executed")
print(f"Brazil YES: {market_state['Brazil']['yes_price']*100:.1f}%")


In [ ]:
print("\n📅 DAY 3: Argentina Gets Support + Spain NO Bets")
print("-" * 60)

day = 3
timestamp = base_time + timedelta(days=day)

execute_trade('Dave', 'Argentina', 'YES', 200, market_state['Argentina']['yes_price'], timestamp)
execute_trade('Eve', 'Argentina', 'YES', 180, market_state['Argentina']['yes_price'], timestamp)
# Henry bets NO on Spain (doesn't think they'll win)
execute_trade('Henry', 'Spain', 'NO', 100, market_state['Spain']['no_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 3 trades executed")
print(f"Argentina YES: {market_state['Argentina']['yes_price']*100:.1f}%")
print(f"Spain NO: {market_state['Spain']['no_price']*100:.1f}%")


In [ ]:
print("\n📅 DAY 5: France Believers + Germany NO Bets")
print("-" * 60)

day = 5
timestamp = base_time + timedelta(days=day)

execute_trade('Frank', 'France', 'YES', 120, market_state['France']['yes_price'], timestamp)
execute_trade('Grace', 'France', 'YES', 100, market_state['France']['yes_price'], timestamp)
# Iris hedges with NO on Germany
execute_trade('Iris', 'Germany', 'NO', 80, market_state['Germany']['no_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 3 trades executed")
print(f"France YES: {market_state['France']['yes_price']*100:.1f}%")
print(f"Germany NO: {market_state['Germany']['no_price']*100:.1f}%")


In [ ]:
print("\n📅 DAY 7: Germany Support + Argentina NO Bets")
print("-" * 60)

day = 7
timestamp = base_time + timedelta(days=day)

execute_trade('Henry', 'Germany', 'YES', 150, market_state['Germany']['yes_price'], timestamp)
# Jack bets NO on Argentina
execute_trade('Jack', 'Argentina', 'NO', 90, market_state['Argentina']['no_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 2 trades executed")
print(f"Germany YES: {market_state['Germany']['yes_price']*100:.1f}%")
print(f"Argentina NO: {market_state['Argentina']['no_price']*100:.1f}%")


In [ ]:
print("\n📅 DAY 10: Spain Dark Horse + France NO Bets")
print("-" * 60)

day = 10
timestamp = base_time + timedelta(days=day)

execute_trade('Iris', 'Spain', 'YES', 90, market_state['Spain']['yes_price'], timestamp)
execute_trade('Jack', 'Spain', 'YES', 110, market_state['Spain']['yes_price'], timestamp)
# Bob hedges with France NO
execute_trade('Bob', 'France', 'NO', 70, market_state['France']['no_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 3 trades executed")
print(f"Spain YES: {market_state['Spain']['yes_price']*100:.1f}%")
print(f"France NO: {market_state['France']['no_price']*100:.1f}%")


In [ ]:
print("\n📅 DAY 15: More Brazil + Germany NO Bets")
print("-" * 60)

day = 15
timestamp = base_time + timedelta(days=day)

execute_trade('Alice', 'Brazil', 'YES', 50, market_state['Brazil']['yes_price'], timestamp)
execute_trade('Carol', 'Brazil', 'YES', 70, market_state['Brazil']['yes_price'], timestamp)
# Carol also bets NO on Germany (hedge)
execute_trade('Carol', 'Germany', 'NO', 60, market_state['Germany']['no_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 3 trades executed")
print(f"Brazil YES: {market_state['Brazil']['yes_price']*100:.1f}%")
print(f"Germany NO: {market_state['Germany']['no_price']*100:.1f}%")


In [ ]:
print("\n📅 DAY 20: Cross-Market Hedging")
print("-" * 60)

day = 20
timestamp = base_time + timedelta(days=day)

execute_trade('Bob', 'Argentina', 'YES', 60, market_state['Argentina']['yes_price'], timestamp)
execute_trade('Dave', 'France', 'YES', 80, market_state['France']['yes_price'], timestamp)
# Alice bets NO on Argentina
execute_trade('Alice', 'Argentina', 'NO', 50, market_state['Argentina']['no_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 3 trades executed")
print(f"Argentina YES: {market_state['Argentina']['yes_price']*100:.1f}%")
print(f"Argentina NO: {market_state['Argentina']['no_price']*100:.1f}%")


In [ ]:
print("\n📅 DAY 25: Final Positions")
print("-" * 60)

day = 25
timestamp = base_time + timedelta(days=day)

execute_trade('Eve', 'Brazil', 'YES', 100, market_state['Brazil']['yes_price'], timestamp)
execute_trade('Frank', 'Argentina', 'YES', 90, market_state['Argentina']['yes_price'], timestamp)
execute_trade('Grace', 'Germany', 'YES', 70, market_state['Germany']['yes_price'], timestamp)
# Dave bets NO on Spain
execute_trade('Dave', 'Spain', 'NO', 100, market_state['Spain']['no_price'], timestamp)

record_prices(timestamp)

print(f"\n✅ 4 trades executed")
print(f"Final positions locked in!")


In [ ]:
print("\n📅 DAY 30: Market Close")
print("=" * 60)

day = 30
timestamp = base_time + timedelta(days=day)
record_prices(timestamp)

print(f"\n🔒 Market closes for trading")

total_volume = sum(market_state[c]['volume'] for c in COUNTRIES)
print(f"\n✅ Total trades executed: {len(trades)}")
print(f"✅ Total volume: ${total_volume:.2f}")

# Show final market state
print(f"\n📊 Final Market State:")
print("-" * 60)
for country in COUNTRIES:
    yes_prob = market_state[country]['yes_price'] * 100
    no_prob = market_state[country]['no_price'] * 100
    volume = market_state[country]['volume']
    print(f"{country:<12} YES: {yes_prob:>5.1f}%  |  NO: {no_prob:>5.1f}%  |  Vol: ${volume:>7.2f}")

total_yes_prob = sum(market_state[c]['yes_price'] for c in COUNTRIES) * 100
print(f"\nTotal YES probability: {total_yes_prob:.1f}%")


In [4]:
# Display final state before resolution
print("\n📊 Final Market State (GROUPED_BINARY):")
print("=" * 60)
print(f"{'Market':<15} {'YES Price':>12} {'NO Price':>12} {'Volume':>10}")
print("-" * 60)

for country in COUNTRIES:
    yes_prob = market_state[country]['yes_price'] * 100
    no_prob = market_state[country]['no_price'] * 100
    volume = market_state[country]['volume']
    print(f"{country:<15} {yes_prob:>11.1f}% {no_prob:>11.1f}% ${volume:>8.2f}")

total_yes_prob = sum(market_state[c]['yes_price'] for c in COUNTRIES) * 100
print("-" * 60)
print(f"{'TOTAL YES':<15} {total_yes_prob:>11.1f}%")
print(f"\n⚠️  This should be ~100%, not {total_yes_prob:.0f}%!")


📊 Final Market State (GROUPED_BINARY):
Market             YES Price     NO Price     Volume
------------------------------------------------------------
Brazil                 55.8%        44.2% $  287.60
Argentina              54.2%        45.8% $  337.99
France                 52.4%        47.6% $  185.89
Germany                50.8%        49.2% $  178.45
Spain                  50.0%        50.0% $  197.99
------------------------------------------------------------
TOTAL YES             263.1%

⚠️  This should be ~100%, not 263%!


## 3. Arbitrage Check - GROUPED_BINARY Analysis

The hybrid bot checks the **sum of YES probabilities** across all markets:
- If sum > 105%: **SELL opportunity** (markets are overpriced)
- If sum < 95%: **BUY opportunity** (markets are underpriced)
- Otherwise: No arbitrage

In GROUPED_BINARY markets starting at 50% each, we expect a **massive arbitrage** signal initially!

In [5]:
# Check arbitrage (this is what the hybrid bot does)
total_yes_probability = sum(market_state[c]['yes_price'] for c in COUNTRIES)
upper_threshold = 1.05  # 105%
lower_threshold = 0.95  # 95%

print("\n🤖 Hybrid Bot - Arbitrage Analysis (GROUPED_BINARY)")
print("=" * 60)
print(f"Sum of YES Probabilities: {total_yes_probability:.4f} ({total_yes_probability*100:.1f}%)")
print(f"Upper Threshold: {upper_threshold:.4f} ({upper_threshold*100:.0f}%)")
print(f"Lower Threshold: {lower_threshold:.4f} ({lower_threshold*100:.0f}%)")
print()

if total_yes_probability > upper_threshold:
    print("⚡ ARBITRAGE DETECTED: SELL OPPORTUNITY")
    print("   Markets are OVERPRICED")
    print("   Bot Strategy: Sell YES shares across all markets")
    print("   OR: Bet on NO outcomes to profit from mispricing")
    arbitrage = 'SELL'

    # Calculate potential arbitrage profit
    excess_probability = total_yes_probability - 1.0
    print(f"\n   Excess Probability: {excess_probability*100:.1f}%")
    print(f"   Potential profit per $100 invested: ${excess_probability*100:.2f}")

elif total_yes_probability < lower_threshold:
    print("⚡ ARBITRAGE DETECTED: BUY OPPORTUNITY")
    print("   Markets are UNDERPRICED")
    print("   Bot Strategy: Buy YES shares across all markets")
    arbitrage = 'BUY'
else:
    print("✅ NO ARBITRAGE")
    print("   Market is within efficient range")
    print("   Bot will provide normal market making")
    arbitrage = None

deviation = abs(total_yes_probability - 1.0) * 100
print(f"\nDeviation from 100%: {deviation:.1f}%")

if arbitrage:
    print("\n⚠️  This is why GROUPED_BINARY markets need careful initialization!")
    print("    Starting all YES at 50% creates artificial arbitrage.")
    print("    Solution: Use smarter initial probabilities that sum to ~100%")


🤖 Hybrid Bot - Arbitrage Analysis (GROUPED_BINARY)
Sum of YES Probabilities: 2.6309 (263.1%)
Upper Threshold: 1.0500 (105%)
Lower Threshold: 0.9500 (95%)

⚡ ARBITRAGE DETECTED: SELL OPPORTUNITY
   Markets are OVERPRICED
   Bot Strategy: Sell YES shares across all markets
   OR: Bet on NO outcomes to profit from mispricing

   Excess Probability: 163.1%
   Potential profit per $100 invested: $163.09

Deviation from 100%: 163.1%

⚠️  This is why GROUPED_BINARY markets need careful initialization!
    Starting all YES at 50% creates artificial arbitrage.
    Solution: Use smarter initial probabilities that sum to ~100%


## 4. Market Resolution

⚽ **World Cup Final Result: Brazil wins!** 🇧🇷🏆

Resolution for GROUPED_BINARY:
- **Brazil market**: "Will Brazil win?" → **YES** ✅
  - Brazil YES holders get $1.00/share (minus 4% fee)
  - Brazil NO holders get $0
- **Argentina market**: "Will Argentina win?" → **NO** ❌
  - Argentina NO holders get $1.00/share (minus 4% fee)
  - Argentina YES holders get $0
- **France market**: "Will France win?" → **NO** ❌
  - France NO holders get $1.00/share (minus 4% fee)
  - France YES holders get $0
- **Germany market**: "Will Germany win?" → **NO** ❌
  - Germany NO holders get $1.00/share (minus 4% fee)
  - Germany YES holders get $0
- **Spain market**: "Will Spain win?" → **NO** ❌
  - Spain NO holders get $1.00/share (minus 4% fee)
  - Spain YES holders get $0

In [6]:
# Market resolves
WINNER = 'Brazil'

print(f"\n🏆 WORLD CUP 2026 FINAL RESULT: {WINNER} WINS!")
print("=" * 70)
print(f"\nResolution for GROUPED_BINARY markets:")
for country in COUNTRIES:
    if country == WINNER:
        print(f"  ✅ '{country} wins?' → YES (YES holders get paid, NO holders get $0)")
    else:
        print(f"  ❌ '{country} wins?' → NO (NO holders get paid, YES holders get $0)")

# Distribution tracking
all_payouts = []
total_gross_payout = 0
total_fees_collected = 0
total_net_payout = 0

print(f"\n💰 Payout Distribution:\n")

# 1. Pay Brazil YES holders
brazil_yes_shares = sum(balances[trader]['shares'][f'{WINNER}_YES'] for trader in TRADERS)
print(f"📊 {WINNER} YES holders:")
print("=" * 70)
print(f"{'Trader':<12} {'Shares':>10} {'Gross':>12} {'Fee (4%)':>12} {'Net Payout':>12}")
print("-" * 70)

for trader in TRADERS:
    shares = balances[trader]['shares'][f'{WINNER}_YES']

    if shares > 0:
        gross_payout = shares * 1.0
        fee = gross_payout * WINNER_FEE_RATE
        net_payout = gross_payout - fee

        balances[trader]['cash'] += net_payout
        balances[trader]['shares'][f'{WINNER}_YES'] = 0

        # Fee distribution
        platform_fee = fee * PLATFORM_FEE_SHARE
        creator_fee = fee * CREATOR_FEE_SHARE
        balances[PLATFORM]['cash'] += platform_fee
        balances[CREATOR]['cash'] += creator_fee

        all_payouts.append({
            'trader': trader,
            'market': WINNER,
            'side': 'YES',
            'shares': shares,
            'gross': gross_payout,
            'fee': fee,
            'net': net_payout,
        })

        total_gross_payout += gross_payout
        total_fees_collected += fee
        total_net_payout += net_payout

        print(f"{trader:<12} {shares:>10.2f} ${gross_payout:>10.2f} ${fee:>10.2f} ${net_payout:>10.2f}")

print("-" * 70)
print(f"Subtotal: {brazil_yes_shares:.2f} shares → ${total_gross_payout:.2f} gross, ${total_net_payout:.2f} net\n")

# 2. Pay NO holders on losing markets
for country in COUNTRIES:
    if country != WINNER:
        country_no_shares = sum(balances[trader]['shares'][f'{country}_NO'] for trader in TRADERS)

        if country_no_shares > 0:
            print(f"\n📊 {country} NO holders ('{country} wins?' → NO):")
            print("=" * 70)
            print(f"{'Trader':<12} {'Shares':>10} {'Gross':>12} {'Fee (4%)':>12} {'Net Payout':>12}")
            print("-" * 70)

            subtotal_gross = 0
            subtotal_net = 0

            for trader in TRADERS:
                shares = balances[trader]['shares'][f'{country}_NO']

                if shares > 0:
                    gross_payout = shares * 1.0
                    fee = gross_payout * WINNER_FEE_RATE
                    net_payout = gross_payout - fee

                    balances[trader]['cash'] += net_payout
                    balances[trader]['shares'][f'{country}_NO'] = 0

                    # Fee distribution
                    platform_fee = fee * PLATFORM_FEE_SHARE
                    creator_fee = fee * CREATOR_FEE_SHARE
                    balances[PLATFORM]['cash'] += platform_fee
                    balances[CREATOR]['cash'] += creator_fee

                    all_payouts.append({
                        'trader': trader,
                        'market': country,
                        'side': 'NO',
                        'shares': shares,
                        'gross': gross_payout,
                        'fee': fee,
                        'net': net_payout,
                    })

                    total_gross_payout += gross_payout
                    total_fees_collected += fee
                    total_net_payout += net_payout
                    subtotal_gross += gross_payout
                    subtotal_net += net_payout

                    print(f"{trader:<12} {shares:>10.2f} ${gross_payout:>10.2f} ${fee:>10.2f} ${net_payout:>10.2f}")

            print("-" * 70)
            print(f"Subtotal: {country_no_shares:.2f} shares → ${subtotal_gross:.2f} gross, ${subtotal_net:.2f} net")

# Clear all remaining shares
for trader in TRADERS:
    for country in COUNTRIES:
        balances[trader]['shares'][f'{country}_YES'] = 0
        balances[trader]['shares'][f'{country}_NO'] = 0

print(f"\n\n{'='*70}")
print(f"💵 GRAND TOTAL:")
print(f"  Total Winning Shares: {sum(p['shares'] for p in all_payouts):.2f}")
print(f"  Gross Payout: ${total_gross_payout:.2f}")
print(f"  Total Fees (4%): ${total_fees_collected:.2f}")
print(f"  Net to Winners: ${total_net_payout:.2f}")

print(f"\n💵 Fee Distribution:")
print(f"  Platform ({PLATFORM}): ${balances[PLATFORM]['cash']:.2f} (2%)")
print(f"  Creator ({CREATOR}): ${balances[CREATOR]['cash']:.2f} (2%)")

print(f"\n❌ Losing Positions (get $0):")
print(f"  {WINNER} NO holders: $0")
for country in COUNTRIES:
    if country != WINNER:
        yes_losers = sum(trade['shares'] for trade in trades if trade['country'] == country and trade['side'] == 'YES')
        if yes_losers > 0:
            print(f"  {country} YES holders: $0")


🏆 WORLD CUP 2026 FINAL RESULT: Brazil WINS!

Resolution for GROUPED_BINARY markets:
  ✅ 'Brazil wins?' → YES (YES holders get paid, NO holders get $0)
  ❌ 'Argentina wins?' → NO (NO holders get paid, YES holders get $0)
  ❌ 'France wins?' → NO (NO holders get paid, YES holders get $0)
  ❌ 'Germany wins?' → NO (NO holders get paid, YES holders get $0)
  ❌ 'Spain wins?' → NO (NO holders get paid, YES holders get $0)

💰 Payout Distribution:

📊 Brazil YES holders:
Trader           Shares        Gross     Fee (4%)   Net Payout
----------------------------------------------------------------------
Alice            150.00 $    150.00 $      6.00 $    144.00
Bob              150.00 $    150.00 $      6.00 $    144.00
Carol            150.00 $    150.00 $      6.00 $    144.00
Eve              100.00 $    100.00 $      4.00 $     96.00
----------------------------------------------------------------------
Subtotal: 550.00 shares → $550.00 gross, $528.00 net


📊 Argentina NO holders ('Argentina

## 5. Final Results & Profit/Loss Analysis

In [7]:
# Calculate P&L for each trader
pnl_results = []

print("\n📈 Trader Profit & Loss Summary")
print("=" * 70)
print(f"{'Trader':<12} {'Initial':>10} {'Final Cash':>12} {'Spent':>10} {'P&L':>10} {'ROI':>8}")
print("-" * 70)

for trader in TRADERS:
    initial_cash = 1000
    final_cash = balances[trader]['cash']

    # Calculate total spent
    spent = sum(trade['cost'] for trade in trades if trade['trader'] == trader)

    # P&L
    pnl = final_cash - initial_cash
    roi = (pnl / initial_cash) * 100 if initial_cash > 0 else 0

    pnl_results.append({
        'trader': trader,
        'initial': initial_cash,
        'final': final_cash,
        'spent': spent,
        'pnl': pnl,
        'roi': roi
    })

    pnl_str = f"${pnl:+.2f}"
    roi_str = f"{roi:+.1f}%"

    print(f"{trader:<12} ${initial_cash:>8.2f} ${final_cash:>10.2f} ${spent:>8.2f} {pnl_str:>10} {roi_str:>8}")

print("-" * 70)

# Overall statistics
winners_count = sum(1 for r in pnl_results if r['pnl'] > 0)
losers_count = len(TRADERS) - winners_count

print(f"\n📊 Market Statistics:")
print(f"  Winners: {winners_count} traders")
print(f"  Losers: {losers_count} traders")
print(f"  Total Volume: ${total_volume:.2f}")
print(f"  Fees Collected: ${total_fees_collected:.2f}")
print(f"  Platform Revenue: ${balances[PLATFORM]['cash']:.2f}")
print(f"  Creator Revenue: ${balances[CREATOR]['cash']:.2f}")


📈 Trader Profit & Loss Summary
Trader          Initial   Final Cash      Spent        P&L      ROI
----------------------------------------------------------------------
Alice        $ 1000.00 $   1092.15 $   99.85    $+92.15    +9.2%
Bob          $ 1000.00 $   1069.43 $  141.77    $+69.43    +6.9%
Carol        $ 1000.00 $   1092.25 $  109.35    $+92.25    +9.2%
Dave         $ 1000.00 $    905.74 $  190.26    $-94.26    -9.4%
Eve          $ 1000.00 $    947.74 $  148.26    $-52.26    -5.2%
Frank        $ 1000.00 $    892.11 $  107.89   $-107.89   -10.8%
Grace        $ 1000.00 $    913.74 $   86.26    $-86.26    -8.6%
Henry        $ 1000.00 $    972.20 $  123.80    $-27.80    -2.8%
Iris         $ 1000.00 $    992.70 $   84.10     $-7.30    -0.7%
Jack         $ 1000.00 $    990.01 $   96.39     $-9.99    -1.0%
----------------------------------------------------------------------

📊 Market Statistics:
  Winners: 3 traders
  Losers: 7 traders
  Total Volume: $1187.93
  Fees Collected: $4

## 6. Key Insights - GROUPED_BINARY with NO Payouts

### Critical Understanding:

In GROUPED_BINARY markets, when Brazil wins:
- **Brazil market resolves to YES** → Brazil YES holders win
- **Argentina market resolves to NO** → Argentina NO holders win!
- **France market resolves to NO** → France NO holders win!
- **Germany market resolves to NO** → Germany NO holders win!
- **Spain market resolves to NO** → Spain NO holders win!

This is VERY different from Multi-Choice markets where only ONE outcome wins.

### Smart Trading Strategy:
If you're confident Brazil will win, you can:
1. Buy Brazil YES shares
2. ALSO buy NO shares on Argentina, France, Germany, Spain
3. If Brazil wins, you get paid on ALL 5 positions!

### Why This Matters:
- NO holders on losing markets are WINNERS too!
- Platform/Creator earn fees from ALL winners (YES + NO)
- Total payout can exceed the winning market alone